So, we left off the previous notebook with a question - "could we optimize the data processing time?" This is super important to investigate for the following reasons:
- We are using a GPU for training our model. But could we use for processing the data chunks as well? Basically, while the model is training on batches of data there's a high chance that it can remain idle if the training data is available to it (and it can happen). 
- If we can optimize the data processing time, it will definitely result in faster training time as well because the model won't have to wait for the data to come to it. 
- This gives us an opportunity to mximize the resource utilization as well. Because there could be situations where the GPU/CPU is not being utilized fully. So while the model is training, in the background we could run some threads to fetch the next batch of data.

Let's start the investigation!